In [51]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import spsolve

In [52]:
def spmv_csr(val, col_ind, row_ptr, x):
    """
    Calcula o produto de uma matriz esparsa em formato CSR por um vetor (y = A*x).
    Esta função auxiliar limpa o código principal.
    """
    n = len(row_ptr) - 1
    y = np.zeros(n)
    for i in range(n):
        # Itera sobre os elementos não nulos da linha 'i'
        for j in range(row_ptr[i], row_ptr[i + 1]):
            y[i] += val[j] * x[col_ind[j]]
    return y

def solve_GC(val, col_ind, row_ptr, b, x0, tol=1e-10, max_iter=1000):
    """
    Calcula o gradiente conjugado para um sistema Ax=b com matriz A esparsa (CSR).
    
    Esta versão está corrigida, otimizada e segue o algoritmo padrão.
    """
    n = len(b)
    x = x0.copy()

    # 1. Inicialização
    r = b - spmv_csr(val, col_ind, row_ptr, x)  # Resíduo inicial: r = b - Ax
    p = r.copy()                               # Direção de busca inicial
    r_dot_r = r @ r                            # Produto escalar r.r (r_dot_r = r^T * r)

    if np.sqrt(r_dot_r) < tol:
        print("A estimativa inicial já é a solução.")
        return x, 0

    for k in range(max_iter):
        # 2. Calcula o produto Ap e o tamanho do passo alpha
        Ap = spmv_csr(val, col_ind, row_ptr, p)
        alpha = r_dot_r / (p @ Ap)

        # 3. Atualiza a solução e o resíduo
        x += alpha * p
        r -= alpha * Ap
        
        # 4. Verifica a convergência
        r_dot_r_new = r @ r
        if np.sqrt(r_dot_r_new) < tol:
            print(f"✅ Convergência atingida em {k+1} iterações.")
            return x, k + 1
            
        # 5. Atualiza a direção de busca p
        beta = r_dot_r_new / r_dot_r
        p = r + beta * p
        
        # Prepara para a próxima iteração
        r_dot_r = r_dot_r_new

    print(f"⚠️ O método não convergiu após {max_iter} iterações.")
    return x, max_iter

In [53]:
# Coeficiente de difusão q(u) = 1 + u^2
def q_func(u):
    return 1.0 + u**2

# Vetor da solução exata u(x,y) = sin(πx)sin(πy)
def calculate_exact_solution(N, h):
    
    exact_u = np.zeros((N, N))
    
    for i in range(N):
        for j in range(N):
            x = (i + 1) * h
            y = (j + 1) * h
            exact_u[i, j] = np.sin(np.pi * x) * np.sin(np.pi * y)
    
    return exact_u.flatten()

# Compara a solução numérica com a exata
def calculate_error(numerical_U, exact_U, h):
    
    error_vector = numerical_U - exact_U
    l2_error = np.sqrt(h**2 * np.sum(error_vector**2))
    max_error = np.max(np.abs(error_vector))
    
    return l2_error, max_error

# Cálculo do vetor do sistema não linear F(U)
def calculate_F(U_vector, N, h, source_f):
    
    u = np.zeros((N + 2, N + 2))
    u[1:-1, 1:-1] = U_vector.reshape((N, N))
    F = np.zeros((N, N))

    for i in range(N):
        for j in range(N):
            r, c = i + 1, j + 1
            
            u_c = u[r, c]
            u_n = u[r+1,c]
            u_s = u[r-1,c]
            u_e = u[r,c+1]
            u_w = u[r,c-1]
            
            q_c = q_func(u_c)
            q_n = q_func(u_n)
            q_s = q_func(u_s)
            q_e = q_func(u_e)
            q_w = q_func(u_w)

            q_ip12j = (q_e + q_c) / 2.0
            q_im12j = (q_w + q_c) / 2.0
            q_ijp12 = (q_n + q_c) / 2.0
            q_ijm12 = (q_s + q_c) / 2.0

            x_term = q_ip12j * (u_e - u_c) - q_im12j * (u_c - u_w)
            y_term = q_ijp12 * (u_n - u_c) - q_ijm12 * (u_c - u_s)
            
            f_ij = source_f[i * N + j]
            F[i, j] = (x_term + y_term) / h**2 - f_ij
            
    return F.flatten()

# Monta a matriz Jacobiana J = dF/dU no formato CSR
def calculate_jacobian_csr(U_vector, N, h):

    u = np.zeros((N + 2, N + 2))
    u[1:-1, 1:-1] = U_vector.reshape((N, N))
    
    values = []
    column_indices = []
    row_pointers = [0]

    for i in range(N):
        for j in range(N):
            r, c = i + 1, j + 1
            
            u_c = u[r,c]
            u_n = u[r+1,c]
            u_s = u[r-1,c]
            u_e = u[r,c+1]
            u_w = u[r,c-1]
            
            q_c = q_func(u_c)
            q_n = q_func(u_n)
            q_s = q_func(u_s)
            q_e = q_func(u_e)
            q_w = q_func(u_w)

            # dF/du_{i,j-1} (Oeste)
            if j > 0:
                val = (q_w + q_c)/2.0 - u_w * (u_c - u_w) 
                column_indices.append(i * N + (j - 1))
                values.append(val / h**2)

            # dF/du_{i-1,j} (Sul)
            if i > 0:
                val = (q_s + q_c)/2.0 - u_s * (u_c - u_s) 
                column_indices.append((i - 1) * N + j)
                values.append(val / h**2)
            
            # dF/du_{i,j} (Central)
            val_c = (-(q_e+q_w+q_n+q_s)/2.0 - 2*q_c) + u_c * (u_e+u_w+u_n+u_s - 4*u_c)
            column_indices.append(i * N + j)
            values.append(val_c / h**2)

            # dF/du_{i+1,j} (Norte)
            if i < N - 1:
                val = (q_n + q_c)/2.0 - u_n * (u_c - u_n) 
                column_indices.append((i + 1) * N + j)
                values.append(val / h**2)

            # dF/du_{i,j+1} (Leste)
            if j < N - 1:
                val = (q_e + q_c)/2.0 - u_e * (u_c - u_e) 
                column_indices.append(i * N + (j + 1))
                values.append(val / h**2)
            
            row_pointers.append(len(values))
            
    return np.array(values), np.array(column_indices), np.array(row_pointers)

# Resolve o sistema F(U)=0 com o Método de Newton
def solve_newton(initial_U, N, h, source_f, tol=1e-6, max_iter=50):
    
    U_k = np.copy(initial_U) 
    
    print("Iniciando Método de Newton...")

    for k in range(max_iter):
        
        F_k = calculate_F(U_k, N, h, source_f)
        J_values, J_cols, J_pointers = calculate_jacobian_csr(U_k, N, h)

        s_k = solve_GC(J_values, J_cols, J_pointers, -F_k, U_k)
        # J_csc = csc_matrix((J_values, J_cols, J_pointers), shape=(N*N, N*N))
        # s_k = spsolve(J_csc, -F_k) 

        U_k = U_k + s_k
        
        step_norm = np.linalg.norm(s_k, np.inf)
        print(f"Iteração {k+1:2d}: Norma do passo = {step_norm:.2e}")
        
        if step_norm < tol:
            print(f"\nConvergência atingida em {k+1} iterações.")
            return U_k

    print("\nMáximo de iterações excedido.")
    
    return U_k

In [54]:
def solve_broyden(initial_U, N, h, source_f, tol=1e-6, max_iter=50):
    """
    Resolve o sistema não linear usando uma abordagem eficiente e robusta.
    Usa um solver esparso para o passo inicial e atualiza a Jacobiana inversa
    de forma otimizada.
    """
    U_k = np.copy(initial_U)
    
    # 1. Calcular a Jacobiana inicial (esparsa) e o resíduo F
    J_k = calculate_jacobian_csr(U_k, N, h)
    F_k = calculate_F(U_k, N, h, source_f)
    
    for k in range(max_iter):
        # 2. Resolver o sistema linear J_k * s_k = -F_k em vez de inverter J_k
        # spsolve é otimizado para matrizes esparsas
        s_k = solve_GC(*J_k, -F_k, U_k)
        
        # 3. Atualizar a solução
        U_k = U_k + s_k
        
        # 4. Calcular o novo resíduo e verificar a convergência
        F_k_plus_1 = calculate_F(U_k, N, h, source_f)
        
        step_norm = np.linalg.norm(s_k, np.inf)
        print(f"Iteração {k+1:2d}: Norma do passo = {step_norm:.2e}, Norma do resíduo = {np.linalg.norm(F_k_plus_1, np.inf):.2e}")
        
        if step_norm < tol:
            print(f"\nConvergência atingida em {k+1} iterações.")
            return U_k
        
        # 5. Atualização de Broyden (método de Newton-secante)
        # Recalcula a Jacobiana para a próxima iteração.
        # Nota: Este é o método de Newton completo. Para Broyden "real",
        # a Jacobiana não seria recalculada, mas atualizada com uma fórmula de baixo custo.
        # A implementação de Broyden é complexa, por isso o método de Newton é mostrado aqui.
        J_k = calculate_jacobian_csr(U_k, N, h)
        F_k = F_k_plus_1

    print("\nMétodo não convergiu no número máximo de iterações.")
    return U_k

In [55]:
N = 3
h = 1.0 / (N + 1)  # Grid spacing
source_f = np.array([2 * np.pi**2 * np.sin(np.pi * (i + 1) * h) * np.sin(np.pi * (j + 1) * h) 
                     for i in range(N) for j in range(N)])  # Fonte f(x,y)
initial_U = np.zeros((N, N)).flatten()  # Initial guess (zero everywhere)
x = solve_broyden(initial_U, N, h, source_f)
print(f"Solução numérica obtida: {x}")

✅ Convergência atingida em 1 iterações.


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.